In [1]:
import Levenshtein

In [111]:
import Levenshtein

# "b" "t" "ɡ" "f", "v", "β" "z" "ʃ", "ʒ" "eɪ", "aɪ", "aɪ̃", "ɔɪ"
similar_sounds = [
    # === STOPS (PLOSIVES) ===
    ["p", "pʰ"],                   # p=spin, pʰ=pin
    ["d", "ɾ", "ɾ̃"],              # d=day, ɾ=bu[tt]er AmE tap, ɾ̃=nasalized tap (rapid/assimilated)
    ["k", "kʰ"],                   # k=skate, kʰ=coat
    ["ʔ"],                              # ʔ=uh-oh (glottal stop), BriE bo[tt]le

    # === AFFRICATES ===
    ["tʃ", "dʒ"],                       # tʃ=chop, dʒ=job

    # === FRICATIVES BY PLACE ===
    ["θ", "θʰ", "ð"],                   # θ=thing, θʰ=aspirated θ (careful/exaggerated), ð=this
    ["s", "sʰ"],                   # s=see, sʰ=strongly aspirated s (non-standard)
    ["x", "ɣ"],                         # x=German Ba[ch], ɣ=Spanish a[ɣ]o (lenition; non-English)
    ["h", "ɦ"],                         # h=hat, ɦ=breathy h (Hindi/Czech; non-English)

    # === NASALS (place-matched + syllabic) ===
    ["m", "m̩"],                        # m=map, m̩=botto[m] (syllabic m)
    ["n", "n̩"],                        # n=no, n̩=butto[n] (syllabic n)
    ["ŋ", "ŋ̍"],                        # ŋ=si[ng], ŋ̍=syllabic ŋ (rare; non-English)

    # === LATERALS / RHOTICS / GLIDES ===
    ["l", "l̩"],                        # l=lip, l̩=bott[le] AmE (syllabic l)
    ["ɹ", "r"],                         # ɹ=run (English rhotic), r=trill/tap in other langs (non-English in GenAm)
    
    # === FRONT VOWELS ===
    ["i", "ĩ", "ĩ", "ɪ"],              # i=FLEECE, ĩ/ĩ=nasal i (non-English), ɪ=KIT
    ["e", "ɛ", "ɛ̃"],                   # e=Sp. "pepe" (close-mid; Eng allophone), ɛ=DRESS, ɛ̃=nasal ɛ (Fr. vin; non-English)
    ["æ", "æ̃", "a"],                   # æ=TRAP, æ̃=nasal æ (non-English), a=Spa "casa" (open front; Eng in some accents)

    # === CENTRAL VOWELS ===
    ["ə", "ə̃", "ə̥", "ɜ", "ʌ", "ɨ"],   # ə=commA (schwa), ə̃=nasal schwa, ə̥=devoiced schwa, ɜ=NURSE (BrE non-rhotic), ʌ=STRUT, ɨ=close central (non-English)
    ["ɚ", "ɝ"],                         # ɚ=lett[er] (rhotic schwa), ɝ=b[ir]d (stressed rhotic)

    # === BACK VOWELS ===
    ["u", "ʊ", "ʊ̃", "ʉ"],              # u=GOOSE, ʊ=FOOT, ʊ̃=nasal ʊ (non-English), ʉ=close central rounded (Scots/Swed; near-back)
    ["o", "ɔ", "ɒ"],                    # o=close-mid back rounded (non-English in GenAm), ɔ=THOUGHT (BrE/GenAm~), ɒ=LOT (BrE)
    ["ɑ", "ɑ̃"],                        # ɑ=PALM/SPA, ɑ̃=nasal a (Fr. "sans"; non-English)

    # === DIPHTHONGS ===
    ["oʊ", "oʊ̃", "əʊ", "aʊ"],          # oʊ=GOAT (GenAm), oʊ̃=nasal oʊ (non-English), əʊ=GOAT (BrE), aʊ=MOUTH
]

# Tạo dictionary để tra cứu nhanh
sound_groups = {}
for group in similar_sounds:
    for sound in group:
        sound_groups[sound] = group

semivowels = ["w", "j"]

def are_similar(sound1, sound2):
    """Kiểm tra 2 âm có nằm trong cùng nhóm similar không"""
    if sound1 == sound2:
        return True
    if sound1 in sound_groups and sound2 in sound_groups:
        return sound_groups[sound1] == sound_groups[sound2]
    return False

def is_semivowel_issue(char, position):
    """Kiểm tra có phải lỗi liên quan đến semivowel không (bỏ qua vị trí đầu)"""
    return char in semivowels and position > 0

# Chuỗi phoneme gốc và chuỗi do người dùng phát âm
correct_phoneme = "dʒænjuwɛɹi"
test_phoneme = "tʃɛnjuwɛɹi"

# Lấy danh sách các thao tác cần thiết
opcodes = Levenshtein.opcodes(correct_phoneme, test_phoneme)

print(f"So sánh '{correct_phoneme}' và '{test_phoneme}':")
print(f"Các thao tác cần thực hiện: {opcodes}\n")

# ANSI color codes
GREEN = '\033[92m'   # Màu xanh lá - đúng
YELLOW = '\033[93m'  # Màu vàng - lỗi nhẹ (similar hoặc semivowel)
RED = '\033[91m'     # Màu đỏ - lỗi nặng
RESET = '\033[0m'    # Reset màu

# Tạo chuỗi có màu
colored_output = ""

print("Diễn giải chi tiết:")
for tag, i1, i2, j1, j2 in opcodes:
    if tag == 'equal':
        # Giống nhau -> màu xanh
        colored_output += GREEN + correct_phoneme[i1:i2] + RESET
        print(f"- Giống nhau: '{correct_phoneme[i1:i2]}'")
    
    elif tag == 'replace':
        correct_char = correct_phoneme[i1:i2]
        test_char = test_phoneme[j1:j2]
        
        # Kiểm tra xem có phải similar sound không
        if are_similar(correct_char, test_char):
            colored_output += YELLOW + correct_char + RESET
            print(f"- TƯƠNG TỰ (YELLOW): Ký tự '{correct_char}' (tại vị trí {i1}) được thay bằng '{test_char}' (cùng nhóm âm)")
        else:
            colored_output += RED + correct_char + RESET
            print(f"- THAY THẾ (RED): Ký tự '{correct_char}' (tại vị trí {i1}) bằng '{test_char}'")
    
    elif tag == 'delete':
        correct_char = correct_phoneme[i1:i2]
        
        # Kiểm tra xem có phải semivowel bị xóa không (và không ở vị trí đầu)
        if is_semivowel_issue(correct_char, i1):
            colored_output += YELLOW + correct_char + RESET
            print(f"- THIẾU SEMIVOWEL (YELLOW): Ký tự '{correct_char}' (tại vị trí {i1}) bị thiếu")
        else:
            colored_output += RED + correct_char + RESET
            print(f"- XÓA (RED): Ký tự '{correct_char}' (tại vị trí {i1})")
    
    elif tag == 'insert':
        test_char = test_phoneme[j1:j2]
        
        # Kiểm tra xem có phải semivowel bị thêm vào không (và không ở vị trí đầu)
        if is_semivowel_issue(test_char, i1):
            colored_output += YELLOW + '[' + test_char + ']' + RESET
            print(f"- THÊM SEMIVOWEL (YELLOW): Ký tự '{test_char}' được thêm vào (tại vị trí {i1})")
        else:
            colored_output += RED + '[' + test_char + ']' + RESET
            print(f"- CHÈN (RED): Ký tự '{test_char}' (tại vị trí {i1})")

print(f"\nKết quả hiển thị correct_phoneme với màu:")
print(colored_output)
print(f"Correct phoneme: {correct_phoneme}")
print(f"Test phoneme: {test_phoneme}")
print(f"\nChú thích:")
print(f"{GREEN}Xanh = đúng{RESET}")
print(f"{YELLOW}Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j){RESET}")
print(f"{RED}Đỏ = lỗi nặng{RESET}")

So sánh 'dʒænjuwɛɹi' và 'tʃɛnjuwɛɹi':
Các thao tác cần thực hiện: [('replace', 0, 3, 0, 3), ('equal', 3, 10, 3, 10)]

Diễn giải chi tiết:
- THAY THẾ (RED): Ký tự 'dʒæ' (tại vị trí 0) bằng 'tʃɛ'
- Giống nhau: 'njuwɛɹi'

Kết quả hiển thị correct_phoneme với màu:
dʒænjuwɛɹi
Correct phoneme: dʒænjuwɛɹi
Test phoneme: tʃɛnjuwɛɹi

Chú thích:
Xanh = đúng
Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j)
Đỏ = lỗi nặng


In [10]:
import difflib
import unicodedata
import Levenshtein  # không còn dùng opcodes của lib này, nhưng giữ nếu bạn còn cần distance ở nơi khác

# === dữ liệu nhóm âm GIỮ NGUYÊN như bạn có ===
similar_sounds = [
    # === STOPS (PLOSIVES) ===
    ["p", "pʰ"],
    ["d", "ɾ", "ɾ̃"],
    ["k", "kʰ"],
    ["ʔ"],

    # === AFFRICATES ===
    ["tʃ", "dʒ"],

    # === FRICATIVES BY PLACE ===
    ["θ", "θʰ", "ð"],
    ["s", "sʰ"],
    ["x", "ɣ"],
    ["h", "ɦ"],

    # === NASALS (place-matched + syllabic) ===
    ["m", "m̩"],
    ["n", "n̩"],
    ["ŋ", "ŋ̍"],

    # === LATERALS / RHOTICS / GLIDES ===
    ["l", "l̩"],
    ["ɹ", "r"],
    
    # === FRONT VOWELS ===
    ["i", "ĩ", "ĩ", "ɪ"],
    ["e", "ɛ", "ɛ̃"],
    ["æ", "æ̃", "a"],

    # === CENTRAL VOWELS ===
    ["ə", "ə̃", "ə̥", "ɜ", "ʌ", "ɨ"],
    ["ɚ", "ɝ"],

    # === BACK VOWELS ===
    ["u", "ʊ", "ʊ̃", "ʉ"],
    ["o", "ɔ", "ɒ"],
    ["ɑ", "ɑ̃"],

    # === DIPHTHONGS ===
    ["oʊ", "oʊ̃", "əʊ", "aʊ"],
]

# Bảng tra nhanh nhóm âm
sound_groups = {}
for group in similar_sounds:
    for sound in group:
        sound_groups[sound] = group

# Tập token hợp lệ để tokenizer ưu tiên khớp dài nhất
KNOWN_TOKENS = set([s for group in similar_sounds for s in group])
semivowels = ["w", "j"]
KNOWN_TOKENS.update(semivowels)  # thêm bán nguyên âm

def tokenize_ipa(s: str):
    tokens = []
    i = 0
    # khớp dài nhất: 3 -> 2 -> 1
    while i < len(s):
        match = None
        # thử 3 ký tự
        if i + 3 <= len(s) and s[i:i+3] in KNOWN_TOKENS:
            match = s[i:i+3]; i += 3
        # thử 2 ký tự
        elif i + 2 <= len(s) and s[i:i+2] in KNOWN_TOKENS:
            match = s[i:i+2]; i += 2
        else:
            # 1 ký tự: nếu không thuộc known tokens, vẫn lấy làm token đơn lẻ
            match = s[i]; i += 1
        tokens.append(match)
    return tokens

def refine_replace_to_per_token(correct_tokens, test_tokens, i1, i2, j1, j2):
    ops = []
    len_c = i2 - i1
    len_t = j2 - j1
    if len_c == len_t:
        # tách thành từng cặp 1–1
        for k in range(len_c):
            ci = i1 + k
            tj = j1 + k
            if correct_tokens[ci] == test_tokens[tj]:
                ops.append(('equal', ci, ci+1, tj, tj+1))
            else:
                ops.append(('replace', ci, ci+1, tj, tj+1))
    else:
        # không thể 1–1: để lại khối gộp
        ops.append(('replace', i1, i2, j1, j2))
    return ops

def are_similar(sound1, sound2):
    """Kiểm tra 2 âm (đã là token) có thuộc cùng nhóm similar không."""
    if sound1 == sound2:
        return True
    return (sound1 in sound_groups) and (sound2 in sound_groups) and (sound_groups[sound1] is sound_groups[sound2])

def is_semivowel_issue(token, token_index):
    """Xem có phải lỗi liên quan đến bán nguyên âm (w/j) và không ở vị trí đầu."""
    return (token in semivowels) and (token_index > 0)

# ===== Demo so sánh =====
correct_phoneme = "dʒænjuɹi"
test_phoneme    = "tʃɛnjuweɹi"

# Token hóa
correct_tokens = tokenize_ipa(correct_phoneme)
test_tokens    = tokenize_ipa(test_phoneme)

# Lấy opcodes ở MỨC TOKEN (khác với trước đây là mức ký tự)
sm = difflib.SequenceMatcher(a=correct_tokens, b=test_tokens)
opcodes = sm.get_opcodes()
print((opcodes[0][0]))
print(opcodes)

print(f"So sánh '{correct_phoneme}' và '{test_phoneme}':")
print(f"Tokens correct: {correct_tokens}")
print(f"Tokens test   : {test_tokens}")
print(f"Các thao tác cần thực hiện (token-level): {opcodes}\n")

# ANSI color codes
GREEN = '\033[92m'   # đúng
YELLOW = '\033[93m'  # lỗi nhẹ
RED = '\033[91m'     # lỗi nặng
RESET = '\033[0m'

colored_output = ""

refined = []
for tag, i1, i2, j1, j2 in opcodes:
    if tag == 'replace':
        refined.extend(refine_replace_to_per_token(correct_tokens, test_tokens, i1, i2, j1, j2))
    else:
        refined.append((tag, i1, i2, j1, j2))

opcodes = refined

print("Diễn giải chi tiết:")
for tag, i1, i2, j1, j2 in opcodes:
    if tag == 'equal':
        seg = "".join(correct_tokens[i1:i2])
        colored_output += GREEN + seg + RESET
        print(f"- Giống nhau: {correct_tokens[i1:i2]}")
    elif tag == 'replace':
        c_tok = "".join(correct_tokens[i1:i2])
        t_tok = "".join(test_tokens[j1:j2])

        # Nếu số lượng token 2 bên cùng bằng 1, có thể kiểm tra similar theo từng token
        if (i2 - i1 == 1) and (j2 - j1 == 1) and are_similar(correct_tokens[i1], test_tokens[j1]):
            colored_output += YELLOW + c_tok + RESET
            print(f"- TƯƠNG TỰ (YELLOW): '{correct_tokens[i1]}' -> '{test_tokens[j1]}' (cùng nhóm)")
        else:
            colored_output += RED + c_tok + RESET
            print(f"- THAY THẾ (RED): {correct_tokens[i1:i2]} -> {test_tokens[j1:j2]}")
    elif tag == 'delete':
        # token từ correct bị thiếu
        for idx in range(i1, i2):
            tok = correct_tokens[idx]
            if is_semivowel_issue(tok, idx):
                colored_output += YELLOW + tok + RESET
                print(f"- THIẾU SEMIVOWEL (YELLOW): '{tok}' tại vị trí token {idx}")
            else:
                colored_output += RED + tok + RESET
                print(f"- XÓA (RED): '{tok}' tại vị trí token {idx}")
    elif tag == 'insert':
        # token chỉ có ở test
        ins_str = "".join(test_tokens[j1:j2])
        # nếu chèn đúng 1 bán nguyên âm (không ở đầu câu theo vị trí tương ứng i1)
        if (j2 - j1 == 1) and is_semivowel_issue(test_tokens[j1], i1):
            colored_output += YELLOW + '[' + ins_str + ']' + RESET
            print(f"- THÊM SEMIVOWEL (YELLOW): '{ins_str}' tại vị trí (theo correct) {i1}")
        else:
            colored_output += RED + '[' + ins_str + ']' + RESET
            print(f"- CHÈN (RED): {test_tokens[j1:j2]} tại vị trí (theo correct) {i1}")

print(f"\nKết quả hiển thị correct_phoneme với màu:")
print(colored_output)
print(f"Correct phoneme: {correct_phoneme}")
print(f"Test phoneme   : {test_phoneme}")
print(f"\nChú thích:")
print(f"{GREEN}Xanh = đúng{RESET}")
print(f"{YELLOW}Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j){RESET}")
print(f"{RED}Đỏ = lỗi nặng{RESET}")


replace
[('replace', 0, 2, 0, 2), ('equal', 2, 5, 2, 5), ('insert', 5, 5, 5, 7), ('equal', 5, 7, 7, 9)]
So sánh 'dʒænjuɹi' và 'tʃɛnjuweɹi':
Tokens correct: ['dʒ', 'æ', 'n', 'j', 'u', 'ɹ', 'i']
Tokens test   : ['tʃ', 'ɛ', 'n', 'j', 'u', 'w', 'e', 'ɹ', 'i']
Các thao tác cần thực hiện (token-level): [('replace', 0, 2, 0, 2), ('equal', 2, 5, 2, 5), ('insert', 5, 5, 5, 7), ('equal', 5, 7, 7, 9)]

Diễn giải chi tiết:
- TƯƠNG TỰ (YELLOW): 'dʒ' -> 'tʃ' (cùng nhóm)
- THAY THẾ (RED): ['æ'] -> ['ɛ']
- Giống nhau: ['n', 'j', 'u']
- CHÈN (RED): ['w', 'e'] tại vị trí (theo correct) 5
- Giống nhau: ['ɹ', 'i']

Kết quả hiển thị correct_phoneme với màu:
dʒænju[we]ɹi
Correct phoneme: dʒænjuɹi
Test phoneme   : tʃɛnjuweɹi

Chú thích:
Xanh = đúng
Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j)
Đỏ = lỗi nặng


In [79]:
import Levenshtein

similar_sounds = [
    # === NGUYÊN ÂM ===
    
    # Nguyên âm trước (Front Vowels)
    ["i", "ɪ", "ĩ"],         # "seat" / "sit" / nasal
    ["e", "ɛ", "ɛ̃"],         # "late" (gần) / "let" / nasal
    ["æ", "æ̃", "a"],       # "cat" / "father" (biến thể của 'a') / nasal
    
    # Nguyên âm giữa (Central Vowels)
    ["ə", "ə̃", "ə̥"],        # Schwa (âm lướt) và các biến thể
    ["ɚ", "ɝ"],             # Âm schwa có r (butter) và âm 'ur' (fur)
    ["ʌ", "ɜ"],             # "cup" / "bird" (non-rhotic) - khá gần nhau
    
    # Nguyên âm sau (Back Vowels)
    ["u", "ʊ", "ʊ̃"],       # "pool" / "pull" / nasal - "ʉ" đã bị tách ra
    ["o", "ɔ", "ɒ"],       # "goat" (gần) / "thought" / "lot" (RP)
    ["ɑ", "ɑ̃"],             # "father" / nasal
    
    # Nguyên âm đôi (Diphthongs) - Tách ra vì chúng không giống nhau
    ["eɪ"],                 # "say" (Tách từ nhóm "aɪ", "ɔɪ")
    ["aɪ", "aɪ̃"],         # "my" / nasal (Tách từ nhóm "eɪ", "ɔɪ")
    ["ɔɪ"],                 # "boy" (Tách từ nhóm "eɪ", "aɪ")
    
    ["oʊ", "oʊ̃", "əʊ"],    # "go" (AmE/nasal/BrE) - Tách từ "aʊ"
    ["aʊ"],                 # "now" - (Tách từ nhóm "oʊ")

    # === PHỤ ÂM - NHÓM THEO CẶP NHẦM LẪN (VỊ TRÍ & ĐỘ VANG) ===

    # Phụ âm tắc (Stops) - Nhóm theo cặp vô thanh/hữu thanh
    ["p", "pʰ", "b"],      # Âm môi (P, P-bật hơi, B)
    ["t", "d"],             # Âm lợi/răng (T, D) - (có thể thêm tʰ nếu cần)
    ["k", "kʰ", "ɡ"],      # Âm ngạc mềm (K, K-bật hơi, G)
    ["ʔ"],                  # Âm tắc thanh hầu (Glottal stop)

    # Phụ âm xát (Fricatives) - Nhóm theo cặp vô thanh/hữu thanh
    ["f", "v", "β"],        # Âm môi-răng (F, V) và âm môi (β)
    ["θ", "θʰ", "ð"],      # Âm răng ("think", "this")
    ["s", "sʰ", "z"],      # Âm lợi ("see", "zoo")
    ["ʃ", "ʒ"],             # Âm sau lợi ("she", "measure")
    ["x", "ɣ"],             # Âm ngạc mềm ("loch", "Bach")
    ["h", "ɦ"],             # Âm thanh hầu ("he", "ahead")

    # Phụ âm tắc xát (Affricates)
    ["tʃ", "dʒ"],           # "church" / "judge" (Đây là một cặp tốt)

    # Phụ âm mũi (Nasals) - Tách theo vị trí cấu âm
    ["m", "m̩"],             # Âm môi (M, M-âm tiết)
    ["n", "n̩"],             # Âm lợi (N, N-âm tiết)
    ["ŋ", "ŋ̍"],             # Âm ngạc mềm ("sing", "sing-âm tiết")

    # Âm lỏng (Liquids)
    ["r", "ɹ", "ɾ", "ɾ̃"],  # Các âm 'R' (R-rung, R-tiếp cận, R-vỗ)
    ["l", "l̩"],             # Âm 'L' (L, L-âm tiết)
]

# Tạo dictionary để tra cứu nhanh
sound_groups = {}
for group in similar_sounds:
    for sound in group:
        sound_groups[sound] = group

semivowels = ["w", "j"]

def are_similar(sound1, sound2):
    """Kiểm tra 2 âm có nằm trong cùng nhóm similar không"""
    if sound1 == sound2:
        return True
    if sound1 in sound_groups and sound2 in sound_groups:
        return sound_groups[sound1] == sound_groups[sound2]
    return False

def is_semivowel_issue(char, position):
    """Kiểm tra có phải lỗi liên quan đến semivowel không (bỏ qua vị trí đầu)"""
    return char in semivowels and position > 0

def is_semivowel(char):
    """Kiểm tra xem ký tự có phải semivowel không"""
    return char in semivowels

def get_alignment_score(correct_char, test_char, operation):
    """
    Tính điểm cho một alignment (điểm càng thấp càng tốt)
    
    Ưu tiên:
    1. Equal = 0 điểm (tốt nhất)
    2. Similar replace = 1 điểm (lỗi nhẹ)
    3. Semivowel delete/insert (không ở đầu) = 2 điểm (lỗi nhẹ hơn)
    4. Non-similar replace/delete/insert = 3 điểm (lỗi nặng)
    """
    if operation == 'equal':
        return 0
    elif operation == 'replace':
        if are_similar(correct_char, test_char):
            return 1  # Similar - lỗi nhẹ
        else:
            return 3  # Không similar - lỗi nặng
    elif operation in ['delete', 'insert']:
        char = correct_char if operation == 'delete' else test_char
        if is_semivowel(char):
            return 2  # Semivowel - lỗi nhẹ hơn
        else:
            return 3  # Lỗi nặng
    return 3

def optimize_alignment(correct_phoneme, test_phoneme, edit_operations):
    """
    Tối ưu alignment bằng cách thử shift các đoạn có DELETE/INSERT liên tiếp
    để tìm alignment có tổng điểm thấp nhất (ưu tiên similar sounds)
    """
    operations = list(edit_operations)
    n = len(operations)
    i = 0
    optimized = []
    
    while i < n:
        # Tìm đoạn liên tiếp có chứa DELETE hoặc có REPLACE không similar
        segment_start = i
        segment_end = i
        has_problem = False
        
        # Mở rộng segment để bao gồm các operations liên quan
        while segment_end < n:
            tag, pos_i, pos_j = operations[segment_end]
            
            if tag == 'delete' or tag == 'insert':
                has_problem = True
                segment_end += 1
            elif tag == 'replace':
                correct_char = correct_phoneme[pos_i]
                test_char = test_phoneme[pos_j]
                if not are_similar(correct_char, test_char):
                    has_problem = True
                    segment_end += 1
                else:
                    # Similar replace - có thể dừng hoặc tiếp tục tùy context
                    if has_problem:
                        segment_end += 1
                    else:
                        break
            else:  # equal
                break
        
        # Nếu không có vấn đề gì, giữ nguyên operation này
        if not has_problem or segment_start == segment_end:
            optimized.append(operations[i])
            i += 1
            continue
        
        # Thử shift alignment trong segment này
        segment = operations[segment_start:segment_end]
        best_alignment = segment
        best_score = float('inf')
        
        # Tính điểm cho alignment hiện tại
        current_score = 0
        for tag, pos_i, pos_j in segment:
            if tag == 'equal':
                current_score += 0
            elif tag == 'replace':
                current_score += get_alignment_score(
                    correct_phoneme[pos_i], 
                    test_phoneme[pos_j], 
                    'replace'
                )
            elif tag == 'delete':
                current_score += get_alignment_score(
                    correct_phoneme[pos_i], 
                    '', 
                    'delete'
                )
            elif tag == 'insert':
                current_score += get_alignment_score(
                    '', 
                    test_phoneme[pos_j], 
                    'insert'
                )
        
        # Thử shift sang phải (DELETE đầu tiên thay vì REPLACE)
        if len(segment) >= 2:
            first_tag, first_i, first_j = segment[0]
            
            if first_tag == 'replace':
                first_correct = correct_phoneme[first_i]
                
                # Nếu ký tự đầu là semivowel, thử chuyển thành DELETE
                if is_semivowel(first_correct):
                    # Tạo alignment mới: shift tất cả test indices sang trái
                    new_segment = [('delete', first_i, first_j)]
                    new_score = get_alignment_score(first_correct, '', 'delete')
                    
                    for k in range(1, len(segment)):
                        tag, pos_i, pos_j = segment[k]
                        if tag == 'replace':
                            # Shift index của test sang trái
                            new_test_idx = pos_j - 1 if k == 1 else segment[k-1][2]
                            new_segment.append(('replace', pos_i, new_test_idx))
                            new_score += get_alignment_score(
                                correct_phoneme[pos_i],
                                test_phoneme[new_test_idx] if new_test_idx < len(test_phoneme) else '',
                                'replace'
                            )
                        elif tag == 'delete':
                            # DELETE trở thành REPLACE với test char trước đó
                            prev_test_idx = segment[k-1][2] if k > 0 else first_j
                            if prev_test_idx < len(test_phoneme):
                                new_segment.append(('replace', pos_i, prev_test_idx))
                                new_score += get_alignment_score(
                                    correct_phoneme[pos_i],
                                    test_phoneme[prev_test_idx],
                                    'replace'
                                )
                    
                    if new_score < best_score:
                        best_score = new_score
                        best_alignment = new_segment
        
        # Sử dụng alignment tốt nhất
        optimized.extend(best_alignment if best_score < current_score else segment)
        i = segment_end
    
    return optimized

# Chuỗi phoneme gốc và chuỗi do người dùng phát âm
correct_phoneme = "aɪdɪdəntduɪt"
test_phoneme = "aɪdɪɹəntduɪt"

# Lấy danh sách các thao tác ở mức ký tự đơn
edit_operations = Levenshtein.editops(correct_phoneme, test_phoneme)

print(f"So sánh '{correct_phoneme}' và '{test_phoneme}':")
print(f"Các thao tác GỐC: {edit_operations}")

# Tối ưu lại alignment
optimized_operations = optimize_alignment(correct_phoneme, test_phoneme, edit_operations)
print(f"Các thao tác SAU TỐI ƯU: {optimized_operations}\n")

# ANSI color codes
GREEN = '\033[92m'   # Màu xanh lá - đúng
YELLOW = '\033[93m'  # Màu vàng - lỗi nhẹ (similar hoặc semivowel)
RED = '\033[91m'     # Màu đỏ - lỗi nặng
RESET = '\033[0m'    # Reset màu

# Tạo mảng màu cho từng ký tự trong correct_phoneme
color_map = [GREEN] * len(correct_phoneme)
explanations = []

# Phân tích từng phép toán
for tag, i, j in optimized_operations:
    correct_char = correct_phoneme[i] if i < len(correct_phoneme) else ''
    test_char = test_phoneme[j] if j < len(test_phoneme) else ''
    
    print(f"Processing {correct_char} and {test_char} at positions {i}, {j} with tag {tag}")
    
    if tag == 'replace':
        # Kiểm tra xem có phải similar sound không
        if are_similar(correct_char, test_char):
            color_map[i] = YELLOW
            explanations.append(f"- Vị trí {i}: TƯƠNG TỰ (YELLOW) - '{correct_char}' → '{test_char}' (cùng nhóm âm)")
        else:
            color_map[i] = RED
            explanations.append(f"- Vị trí {i}: THAY THẾ (RED) - '{correct_char}' → '{test_char}'")
    
    elif tag == 'delete':
        # Kiểm tra xem có phải semivowel bị xóa không
        if is_semivowel_issue(correct_char, i):
            color_map[i] = YELLOW
            explanations.append(f"- Vị trí {i}: THIẾU SEMIVOWEL (YELLOW) - '{correct_char}' bị thiếu")
        else:
            color_map[i] = RED
            explanations.append(f"- Vị trí {i}: XÓA (RED) - '{correct_char}' bị xóa")
    
    elif tag == 'insert':
        # Insert không ảnh hưởng đến correct_phoneme, nhưng ta vẫn ghi nhận
        if is_semivowel_issue(test_char, i):
            explanations.append(f"- Vị trí {i}: THÊM SEMIVOWEL (YELLOW) - '{test_char}' được thêm vào (không cần thiết)")
        else:
            explanations.append(f"- Vị trí {i}: CHÈN (RED) - '{test_char}' được thêm vào (sai)")

# In diễn giải chi tiết
print("Diễn giải chi tiết:")
for exp in explanations:
    print(exp)

# In kết quả hiển thị với màu
colored_output = ""
for i, char in enumerate(correct_phoneme):
    colored_output += color_map[i] + char + RESET

print(f"\nKết quả hiển thị correct_phoneme với màu:")
print(colored_output)
print(correct_phoneme)
print(test_phoneme)

# Tính điểm
green_count = sum(1 for c in color_map if c == GREEN)
yellow_count = sum(1 for c in color_map if c == YELLOW)
red_count = sum(1 for c in color_map if c == RED)

print(f"\nThống kê:")
print(f"- Đúng (GREEN): {green_count}/{len(correct_phoneme)}")
print(f"- Lỗi nhẹ (YELLOW): {yellow_count}/{len(correct_phoneme)}")
print(f"- Lỗi nặng (RED): {red_count}/{len(correct_phoneme)}")

accuracy = (green_count + yellow_count * 0.5) / len(correct_phoneme) * 100
print(f"- Độ chính xác: {accuracy:.1f}%")

print(f"\nChú thích:")
print(f"{GREEN}Xanh = đúng{RESET}")
print(f"{YELLOW}Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j){RESET}")
print(f"{RED}Đỏ = lỗi nặng{RESET}")

So sánh 'aɪdɪdəntduɪt' và 'aɪdɪɹəntduɪt':
Các thao tác GỐC: [('replace', 4, 4)]
Các thao tác SAU TỐI ƯU: [('replace', 4, 4)]

Processing d and ɹ at positions 4, 4 with tag replace
Diễn giải chi tiết:
- Vị trí 4: THAY THẾ (RED) - 'd' → 'ɹ'

Kết quả hiển thị correct_phoneme với màu:
aɪdɪdəntduɪt
aɪdɪdəntduɪt
aɪdɪɹəntduɪt

Thống kê:
- Đúng (GREEN): 11/12
- Lỗi nhẹ (YELLOW): 0/12
- Lỗi nặng (RED): 1/12
- Độ chính xác: 91.7%

Chú thích:
Xanh = đúng
Vàng = lỗi nhẹ (âm tương tự hoặc thiếu/thêm semivowel w/j)
Đỏ = lỗi nặng


In [4]:
import Levenshtein
import json

def compare_phonemes(correct_phoneme, test_phoneme):
    """
    So sánh 2 chuỗi phoneme và trả về kết quả dạng JSON cho FE
    """
    # Lấy danh sách các thao tác cần thiết
    opcodes = Levenshtein.opcodes(correct_phoneme, test_phoneme)
    
    # Tạo danh sách kết quả cho từng ký tự
    result = []
    
    for tag, i1, i2, j1, j2 in opcodes:
        if tag == 'equal':
            # Giống nhau -> màu xanh
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "correct",
                    "color": "green"
                })
        elif tag == 'replace':
            # Thay thế -> màu đỏ
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "wrong",
                    "color": "red",
                    "replaced_with": test_phoneme[j1 + (i - i1)] if j1 + (i - i1) < j2 else None
                })
        elif tag == 'delete':
            # Xóa -> màu đỏ
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "delete",
                    "color": "red"
                })
        elif tag == 'insert':
            # Chèn -> không hiển thị trong correct_phoneme nhưng ghi nhận
            pass
    
    # Tính điểm chính xác
    total_chars = len(correct_phoneme)
    correct_chars = sum(1 for item in result if item["status"] == "correct")
    accuracy = round((correct_chars / total_chars * 100), 2) if total_chars > 0 else 0
    
    return {
        "correct_phoneme": correct_phoneme,
        "test_phoneme": test_phoneme,
        "accuracy": accuracy,
        "details": result,
        "summary": {
            "total": total_chars,
            "correct": correct_chars,
            "errors": total_chars - correct_chars
        }
    }


# Test code
if __name__ == "__main__":
    correct_phoneme = "dʒænjuɛɹi"
    test_phoneme = "tʃæɹuɝi"
    
    result = compare_phonemes(correct_phoneme, test_phoneme)
    
    # In kết quả dạng JSON đẹp
    print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "correct_phoneme": "dʒænjuɛɹi",
  "test_phoneme": "tʃæɹuɝi",
  "accuracy": 33.33,
  "details": [
    {
      "char": "d",
      "position": 0,
      "status": "wrong",
      "color": "red",
      "replaced_with": "t"
    },
    {
      "char": "ʒ",
      "position": 1,
      "status": "wrong",
      "color": "red",
      "replaced_with": "ʃ"
    },
    {
      "char": "æ",
      "position": 2,
      "status": "correct",
      "color": "green"
    },
    {
      "char": "n",
      "position": 3,
      "status": "wrong",
      "color": "red",
      "replaced_with": "ɹ"
    },
    {
      "char": "j",
      "position": 4,
      "status": "delete",
      "color": "red"
    },
    {
      "char": "u",
      "position": 5,
      "status": "correct",
      "color": "green"
    },
    {
      "char": "ɛ",
      "position": 6,
      "status": "wrong",
      "color": "red",
      "replaced_with": "ɝ"
    },
    {
      "char": "ɹ",
      "position": 7,
      "status": "delete",
      "color"